In [1]:
from utils import *

In [2]:
train = load_pickle('../train_labeled_0430.pickle')
test = load_pickle('../test_labeled_0430.pickle')

In [5]:
from konlpy.tag import Twitter
twitter = Twitter()

In [12]:
train[0]

(['"""미소가 찾아갈 친구에겐 불편할 수도 있는 이야기. 하지만 미소에겐, 미소를 받아들인 이에겐, 미소를 지켜보는 관중들에겐 불편함은 없었다. 사랑하는 위스키와 담배를 위해 집을 포기한 미소에게, 나아가 약마저 포기한 미소에게 누가 가벼이 말할 수 있나"""'],
 0)

In [13]:
%%time
train_docs = [(tokenize(row[0][0]), row[1]) for row in train]
test_docs = [(tokenize(row[0][0]), row[1]) for row in test]

In [15]:
len(train_docs), len(test_docs)

(152479, 16940)

In [44]:
from gensim.models import doc2vec
from collections import namedtuple

TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [45]:
%%time
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]
tagged_test_docs = [TaggedDocument(d, [c]) for d, c in test_docs]

CPU times: user 398 ms, sys: 13.9 ms, total: 412 ms
Wall time: 415 ms


In [58]:
model = doc2vec.Doc2Vec(size=300, alpha=0.02, min_alpha=0.02, 
                        workers=4, min_count=10, iter=100, seed=1234)
model.build_vocab(tagged_train_docs)

In [59]:
%%time
model.train(tagged_train_docs, epochs=model.iter, total_examples=model.corpus_count)

CPU times: user 18min 8s, sys: 6min 38s, total: 24min 46s
Wall time: 13min 6s


222309735

In [60]:
model.save('../model/doc2vec_0430_alpha002.model')

In [69]:
%%time
X_train = [model.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags[0] for doc in tagged_train_docs]
print(len(X_train))
print(len(X_train[0]))

152479
300
CPU times: user 31.9 s, sys: 516 ms, total: 32.4 s
Wall time: 32.5 s


In [70]:
%%time
X_test = [model.infer_vector(doc.words) for doc in tagged_test_docs]
y_test = [doc.tags[0] for doc in tagged_test_docs]
print(len(X_test))
print(y_test[0])

16940
0
CPU times: user 3.61 s, sys: 11.8 ms, total: 3.62 s
Wall time: 3.63 s


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

lr_clf = LogisticRegression()

In [78]:
%%time
lr_clf.fit(X_train, y_train)

CPU times: user 26.7 s, sys: 1.08 s, total: 27.8 s
Wall time: 28.6 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [79]:
lr_clf.score(X_test, y_test)

0.5711334120425029

**2018/04/30 02:30**

- scikit-learn classifier 결과 (BaseLine)
    - LogisticRegression: 57% (매우 나쁜 성능)
    - NB는 음수가 들어가면 error 발생 --> scaling 후 시도해야 할 듯
- train data 조정 필요할 듯
- `norm/stem` --> 의미 정보가 상당부분 희석되는 문제 발생 (train set 최대한 깔끔하게 조성 시도해야 할 듯)